# Sequence Read Archive (SRA) Quality Analysis on AWS

## Comprehensive Quality Control for Genomic Sequencing Data

This notebook demonstrates how to:
1. Search and discover SRA datasets using metadata queries
2. Download SRA files and convert them to FASTQ format
3. Perform comprehensive quality control analysis (FastQC-style)
4. Store results in AWS S3 for further analysis

The Sequence Read Archive (SRA) data is stored on AWS as part of the Registry of Open Data. Most SRA data is in the proprietary `.sra` format and requires the SRA Toolkit for conversion to FASTQ.

## ⚠️ Important: Instance Size and Dataset Selection

**Instance Requirements:**
- **ml.t3.medium** (2 vCPU, 4GB RAM, ~$0.05/hour): Recommended for datasets <20MB
- **ml.t3.large** (2 vCPU, 8GB RAM, ~$0.10/hour): For datasets 20-100MB

**Disk Space Warning:**
- SRA files expand **10-15x** when converted to FASTQ
- Example: 20MB .sra → 200-300MB FASTQ files
- Example: 187MB .sra → 2GB FASTQ files
- Ensure your instance has sufficient disk space

**Recommendation:**
- Start with small datasets (<20MB) on ml.t3.medium
- Upgrade to ml.t3.large if needed
- This tutorial uses a 7MB dataset (works on medium)

**Cost Optimization:**
- Stop your notebook instance when not in use
- Delete CloudFormation stack when finished

## Part 1: Setup and Installation

### Install Required Libraries

We'll install:
- `pysradb`: Python package for querying SRA metadata and downloading data
- `matplotlib`: For creating quality control visualizations
- `biopython`: For parsing FASTQ files
- `sra-tools`: NCBI toolkit for working with SRA data (installed via conda)

**Note on Installation Output:**
- The installation may show a warning about conda version (e.g., "A newer version of conda exists"). This is safe to ignore.
- Installation output is condensed to show only the last 20 lines. Full logs are available if troubleshooting is needed.
- Expected installation time: 3-5 minutes

In [ ]:
# Install required packages
# Note: Installation may take 3-5 minutes and will show a conda version warning (safe to ignore)
print('Installing packages... This may take a few minutes.')
!pip install -q pysradb matplotlib biopython 2>&1 | grep -v 'already satisfied' || true
!conda install -c bioconda -y sra-tools 2>&1 | tail -n 20
print('\n✓ Installation complete!')

### Import Libraries

In [ ]:
import os
import gzip
import boto3
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from Bio import SeqIO
from collections import defaultdict, Counter
import subprocess
import pandas as pd

# Suppress tqdm warning from pysradb
import warnings
warnings.filterwarnings('ignore', category=Warning)

from pysradb.sraweb import SRAweb

print("All libraries imported successfully!")
print("Note: You may see a TqdmExperimentalWarning - this is safe to ignore.")

### Configure AWS S3 Storage

**To find your S3 bucket name:**
1. Go to the AWS CloudFormation console
2. Select your stack (e.g., `opendata-sra-notebook-stack`)
3. Click the **Outputs** tab
4. Copy the value from the `BucketName` output
5. Paste it in the cell below

In [ ]:
# Update this with your S3 bucket name from CloudFormation
dest_bucket = 'enter_the_bucket_name_created_through_cloud_formation_here'

# Initialize S3 client
s3 = boto3.client('s3')
print(f"S3 client initialized. Using bucket: {dest_bucket}")

## Part 2: Search and Discover SRA Datasets

We'll use pysradb to search for RNA-seq datasets. For this example, we'll use a small, commonly-used dataset.

### How to Search for Datasets

**Important**: Don't browse S3 buckets directly - they contain millions of files!

Instead, use this workflow:
1. **Search metadata** using pysradb or NCBI SRA Run Selector
2. **Find accession numbers** that match your criteria
3. **Download from S3** using the accession number

This is much faster and more efficient than browsing S3.

In [ ]:
# Demonstrate metadata search workflow

print("Method 1: Search using NCBI SRA Run Selector (Web Interface)")
print("  URL: https://www.ncbi.nlm.nih.gov/Traces/study/")
print("  - Search by organism, study, experiment type")
print("  - Filter by Size (shown in Bytes on the website), date, platform")
print("  - Download accession list")
print()

print("Method 2: Search using pysradb (Python)")
print("  Example: Search for datasets by criteria")
print()

# Initialize pysradb
db = SRAweb()

# For this tutorial, we'll use a pre-selected dataset
print("For this tutorial, we selected: SRR390728")
print("\nSelection criteria:")
print("  ✓ Available in AWS S3")
print("  ✓ Small size (~100MB)")
print("  ✓ E. coli RNA-seq")
print("  ✓ Single-end reads")
print("\nOnce you have an accession number, download from S3:")
print("\nS3 Path Pattern:")
print("  s3://sra-pub-run-odp/sra/{ACCESSION}/{ACCESSION}")
print("  Example: s3://sra-pub-run-odp/sra/SRR390728/SRR390728")
print("\nNote: DRR accessions appear well-populated in S3.")
print("      SRR/ERR accessions may have varying availability.")

### Example Dataset: DRR000019

For this tutorial, we'll use **DRR000019** - a small dataset perfect for learning.

**Dataset info:**
- Size: ~7MB (expands to ~70MB FASTQ)
- Works on: ml.t3.medium
- Conversion time: 1-2 minutes

**How to find your own datasets:**

1. **Go to NCBI SRA Run Selector**: https://www.ncbi.nlm.nih.gov/Traces/study/
2. **Search** for your organism/study (e.g., "E. coli RNA-seq")
3. **Filter by size**: Click "Add filter" → "Size" → Select <50MB
4. **Click on a run**: This shows the accession number (SRR/ERR/DRR prefix)
5. **Copy the accession**: Example: SRR390728, DRR000019
6. **Paste into notebook**: Replace the accession in the cell above

**Recommended dataset sizes:**
- ml.t3.medium: <20MB .sra files
- ml.t3.large: 20-100MB .sra files

**Other small datasets to try:**
- DRR000007 (44MB)
- DRR000019 (7MB) ← Current
- SRR390728 (187MB) - requires ml.t3.large

In [ ]:
# Define the SRA accession number
# Using DRR000019 - a small dataset perfect for tutorials
accession = 'DRR000019'

# Initialize pysradb
db = SRAweb()

# Get metadata for this accession
print(f"Fetching metadata for {accession}...\n")
try:
    metadata = db.sra_metadata(accession)
    if not metadata.empty:
        print(f"Dataset: {accession}")
        print(f"Organism: {metadata['organism_name'].values[0]}")
        print(f"Library Strategy: {metadata['library_strategy'].values[0]}")
        print(f"Platform: {metadata['instrument'].values[0]}")
        print(f"\nFull metadata:")
        display(metadata)
except Exception as e:
    print(f"Metadata query note: {e}")
    print(f"Proceeding with {accession} from AWS S3...")

## Part 3: Download SRA Data from AWS S3 and Convert to FASTQ

Now that we have an accession number, we'll:
1. Download the SRA file from AWS Open Data Registry
2. Convert to FASTQ using `fasterq-dump`
3. Compress and upload to your S3 bucket

**S3 Path Structure (discovered through testing):**
```
s3://sra-pub-run-odp/sra/{ACCESSION}/{ACCESSION}
```

**Example:** SRR390728 → `s3://sra-pub-run-odp/sra/SRR390728/SRR390728`

**Note:** Files have NO .sra extension in S3, and NO prefix directories (SRR/SRR390/etc.)

In [ ]:
# Create a directory for downloads
os.makedirs('sra_data', exist_ok=True)
os.chdir('sra_data')

print(f"Downloading {accession} from AWS S3...")
print("\nUsing AWS Open Data Registry (s3://sra-pub-run-odp/)")

# CORRECT path structure (discovered through testing):
# s3://sra-pub-run-odp/sra/{ACCESSION}/{ACCESSION}
# Note: NO .sra extension, NO prefix directories
s3_path = f"s3://sra-pub-run-odp/sra/{accession}/{accession}"

print(f"S3 Path: {s3_path}")
print("Expected time: 1-5 minutes\n")

# Download from S3
try:
    subprocess.run(
        ['aws', 's3', 'cp', s3_path, f'{accession}.sra', '--no-sign-request'],
        check=True
    )
    print(f"\n✓ Downloaded {accession} from S3")
    
    # Verify file
    if os.path.exists(f"{accession}.sra"):
        file_size = os.path.getsize(f"{accession}.sra") / (1024 * 1024)
        print(f"File size: {file_size:.2f} MB")
    else:
        print("Warning: File not found after download")
    
except subprocess.CalledProcessError as e:
    print(f"\nError downloading from S3: {e}")
    print("\nThis could mean:")
    print("1. The accession doesn't exist in S3")
    print("2. Network connectivity issue")
    print("3. The accession may be in a different bucket")
    print("\nTroubleshooting:")
    print(f"  - Verify accession exists: https://www.ncbi.nlm.nih.gov/sra/{accession}")
    print(f"  - Check S3: aws s3 ls s3://sra-pub-run-odp/sra/{accession}/ --no-sign-request")
    print(f"  - Try different accession (DRR accessions seem well-populated)")
except FileNotFoundError:
    print("\nError: AWS CLI not found.")
    print("Install: pip install awscli")

In [ ]:
# Convert SRA to FASTQ
# The file is a valid NCBI SRA format file (187MB)

print(f"Converting {accession}.sra to FASTQ format...")
print(f"File size: {os.path.getsize(f'{accession}.sra') / (1024*1024):.1f} MB")
print("\nThis may take 10-20 minutes for a 187MB file.")
print("Progress will be shown below...\n")

try:
    # Use fasterq-dump with the accession (it will find the .sra file)
    # -e 2: use 2 threads (more stable than 4 for large files)
    # -p: show progress
    # -t /tmp: use temp directory (helps with space)
    # --split-files: split paired-end reads
    
    result = subprocess.run(
        ['fasterq-dump', accession, '-e', '2', '-p', '-t', '/tmp', '--split-files'],
        check=True,
        capture_output=False,  # Show output in real-time
        text=True
    )
    
    print(f"\n✓ Conversion complete")
    
except subprocess.CalledProcessError as e:
    print(f"\nfasterq-dump failed. Trying fastq-dump (slower but more reliable)...\n")
    
    try:
        # Fallback to fastq-dump
        # --split-files: split paired-end
        # --gzip: compress output
        subprocess.run(
            ['fastq-dump', '--split-files', accession],
            check=True
        )
        print(f"\n✓ Conversion complete using fastq-dump")
        
    except subprocess.CalledProcessError as e2:
        print(f"\nConversion failed: {e2}")
        print("\nPossible causes:")
        print("  - Insufficient memory (file is 187MB, needs ~1-2GB RAM)")
        print("  - Insufficient disk space")
        print("  - SageMaker instance too small (try ml.t3.large)")
        print("\nWorkaround: Use a smaller dataset")
        print("  Try: DRR000019 (only 7MB)")
        raise

# List generated FASTQ files
fastq_files = sorted([f for f in os.listdir('.') if f.endswith('.fastq')])

if fastq_files:
    print(f"\nGenerated FASTQ files:")
    for f in fastq_files:
        size = os.path.getsize(f) / (1024 * 1024)
        print(f"  {f} ({size:.1f} MB)")
    
    # Use first file for analysis
    fastq_file = fastq_files[0]
    print(f"\nWill use {fastq_file} for quality analysis")
    
else:
    print("\nWarning: No FASTQ files generated")
    print("Files in directory:", os.listdir('.'))

In [ ]:
# Compress FASTQ file to save space
print(f"Compressing {fastq_file}...")
subprocess.run(['gzip', fastq_file], check=True)
fastq_gz = f"{fastq_file}.gz"
print(f"✓ Compressed to {fastq_gz}")

In [ ]:
# Upload to S3
s3_key = f"sra_data/{accession}/{fastq_gz}"
print(f"Uploading to S3: s3://{dest_bucket}/{s3_key}")

try:
    s3.upload_file(fastq_gz, dest_bucket, s3_key)
    print("✓ Uploaded to S3")
except Exception as e:
    print(f"Error uploading to S3: {e}")

## Part 4: Quality Control Analysis

We'll perform comprehensive FastQC-style quality control analysis including:
1. Per-base quality scores
2. Per-sequence quality distribution
3. Per-base sequence content
4. GC content distribution
5. Sequence length distribution
6. Sequence duplication levels
7. Overrepresented sequences

Each metric will include Pass/Warn/Fail indicators based on standard quality thresholds.

**Scientific Basis:**
These metrics are based on FastQC, the industry-standard tool for sequencing quality control (Andrews, 2010). FastQC is used by every major sequencing facility worldwide and cited in 10,000+ scientific publications. The quality thresholds are based on:
- Phred quality scores (Ewing & Green, 1998)
- Empirical validation across millions of datasets
- Community consensus (ENCODE, NIH/NCBI guidelines)
- Platform specifications (Illumina, PacBio, etc.)

**References:**
- Andrews, S. (2010). FastQC: A Quality Control Tool for High Throughput Sequence Data. http://www.bioinformatics.babraham.ac.uk/projects/fastqc/
- Ewing, B., & Green, P. (1998). Base-calling of automated sequencer traces using phred. Genome Research, 8(3), 186-194.

### Parse FASTQ and Collect Statistics

In [ ]:
# Parse FASTQ file and collect all statistics
print(f"Parsing {fastq_gz} and collecting statistics...")
print("This may take several minutes for large files.\n")

# Data structures for QC metrics
quality_scores = []  # List of quality score arrays
sequence_lengths = []
gc_contents = []
sequences = []
per_base_content = defaultdict(lambda: defaultdict(int))
sequence_counts = Counter()

# Parse FASTQ
read_count = 0
max_reads = 100000  # Limit for faster processing, remove for full analysis

with gzip.open(fastq_gz, 'rt') as handle:
    for record in SeqIO.parse(handle, 'fastq'):
        read_count += 1
        if read_count > max_reads:
            break
        
        # Quality scores
        quality_scores.append(record.letter_annotations['phred_quality'])
        
        # Sequence length
        seq_len = len(record.seq)
        sequence_lengths.append(seq_len)
        
        # GC content
        gc = (record.seq.count('G') + record.seq.count('C')) / seq_len * 100
        gc_contents.append(gc)
        
        # Per-base content
        for pos, base in enumerate(str(record.seq)):
            per_base_content[pos][base] += 1
        
        # Sequence duplication
        sequence_counts[str(record.seq)] += 1
        
        if read_count % 10000 == 0:
            print(f"Processed {read_count:,} reads...", end='\r')

print(f"\n✓ Parsed {read_count:,} reads")
print(f"Average read length: {np.mean(sequence_lengths):.1f} bp")
print(f"Average GC content: {np.mean(gc_contents):.1f}%")

### QC Metric 1: Per-base Quality Scores

In [ ]:
# Calculate per-base quality statistics
max_length = max(len(q) for q in quality_scores)
per_base_quality = [[] for _ in range(max_length)]

for qual_array in quality_scores:
    for pos, q in enumerate(qual_array):
        per_base_quality[pos].append(q)

# Calculate statistics for each position
positions = list(range(1, len(per_base_quality) + 1))
medians = [np.median(q) for q in per_base_quality]
q25 = [np.percentile(q, 25) for q in per_base_quality]
q75 = [np.percentile(q, 75) for q in per_base_quality]
q10 = [np.percentile(q, 10) for q in per_base_quality]
q90 = [np.percentile(q, 90) for q in per_base_quality]

# Determine pass/warn/fail
median_quality = np.median(medians)
if median_quality >= 28:
    status = "PASS"
    color = "green"
elif median_quality >= 20:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Plot
plt.figure(figsize=(14, 6))
plt.fill_between(positions, q10, q90, alpha=0.3, color='lightblue', label='10-90 percentile')
plt.fill_between(positions, q25, q75, alpha=0.5, color='yellow', label='25-75 percentile')
plt.plot(positions, medians, 'b-', linewidth=2, label='Median')
plt.axhline(y=28, color='green', linestyle='--', alpha=0.5, label='Good quality (Q28)')
plt.axhline(y=20, color='orange', linestyle='--', alpha=0.5, label='Acceptable (Q20)')
plt.xlabel('Position in read (bp)')
plt.ylabel('Quality score')
plt.title(f'Per-base Quality Scores - {status}', color=color, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Status: {status}")
print(f"Median quality score: {median_quality:.1f}")

### QC Metric 2: Per-sequence Quality Distribution

In [ ]:
# Calculate mean quality per sequence
mean_qualities = [np.mean(q) for q in quality_scores]

# Determine status
peak_quality = np.percentile(mean_qualities, 50)
if peak_quality >= 30:
    status = "PASS"
    color = "green"
elif peak_quality >= 20:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Plot
plt.figure(figsize=(12, 6))
plt.hist(mean_qualities, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=30, color='green', linestyle='--', label='Excellent (Q30)')
plt.axvline(x=20, color='orange', linestyle='--', label='Acceptable (Q20)')
plt.xlabel('Mean Sequence Quality (Phred Score)')
plt.ylabel('Number of Sequences')
plt.title(f'Per-sequence Quality Score Distribution - {status}', color=color, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Status: {status}")
print(f"Peak quality: {peak_quality:.1f}")
print(f"Mean quality: {np.mean(mean_qualities):.1f}")

### QC Metric 3: Per-base Sequence Content

In [ ]:
# Calculate percentage of each base at each position
positions = sorted(per_base_content.keys())
a_pct = []
t_pct = []
g_pct = []
c_pct = []

for pos in positions:
    total = sum(per_base_content[pos].values())
    a_pct.append(per_base_content[pos]['A'] / total * 100)
    t_pct.append(per_base_content[pos]['T'] / total * 100)
    g_pct.append(per_base_content[pos]['G'] / total * 100)
    c_pct.append(per_base_content[pos]['C'] / total * 100)

# Check for bias (should be relatively flat)
max_variation = max(np.std(a_pct), np.std(t_pct), np.std(g_pct), np.std(c_pct))
if max_variation < 10:
    status = "PASS"
    color = "green"
elif max_variation < 20:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Plot
plt.figure(figsize=(14, 6))
pos_range = list(range(1, len(positions) + 1))
plt.plot(pos_range, a_pct, 'g-', label='%A', linewidth=2)
plt.plot(pos_range, t_pct, 'r-', label='%T', linewidth=2)
plt.plot(pos_range, g_pct, 'b-', label='%G', linewidth=2)
plt.plot(pos_range, c_pct, 'orange', label='%C', linewidth=2)
plt.xlabel('Position in read (bp)')
plt.ylabel('Percentage (%)')
plt.title(f'Per-base Sequence Content - {status}', color=color, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Status: {status}")
print(f"Max variation: {max_variation:.1f}%")

### QC Metric 4: GC Content Distribution

In [ ]:
# Plot GC content distribution
mean_gc = np.mean(gc_contents)
std_gc = np.std(gc_contents)

# Theoretical normal distribution
x = np.linspace(0, 100, 100)
theoretical = np.exp(-0.5 * ((x - mean_gc) / std_gc) ** 2)
theoretical = theoretical / theoretical.max() * max(np.histogram(gc_contents, bins=50)[0])

# Status based on distribution shape
if std_gc < 10:
    status = "PASS"
    color = "green"
elif std_gc < 15:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Plot
plt.figure(figsize=(12, 6))
plt.hist(gc_contents, bins=50, edgecolor='black', alpha=0.7, label='Observed')
plt.plot(x, theoretical, 'r--', linewidth=2, label='Theoretical')
plt.axvline(x=mean_gc, color='blue', linestyle='--', label=f'Mean: {mean_gc:.1f}%')
plt.xlabel('GC Content (%)')
plt.ylabel('Number of Sequences')
plt.title(f'GC Content Distribution - {status}', color=color, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Status: {status}")
print(f"Mean GC: {mean_gc:.1f}%")
print(f"Std Dev: {std_gc:.1f}%")

### QC Metric 5: Sequence Length Distribution

In [ ]:
# Plot sequence length distribution
length_counts = Counter(sequence_lengths)
unique_lengths = len(length_counts)

# Status: should be uniform for Illumina
if unique_lengths == 1:
    status = "PASS"
    color = "green"
elif unique_lengths <= 3:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Plot
plt.figure(figsize=(12, 6))
lengths = sorted(length_counts.keys())
counts = [length_counts[l] for l in lengths]
plt.bar(lengths, counts, edgecolor='black', alpha=0.7)
plt.xlabel('Sequence Length (bp)')
plt.ylabel('Number of Sequences')
plt.title(f'Sequence Length Distribution - {status}', color=color, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"Status: {status}")
print(f"Unique lengths: {unique_lengths}")
print(f"Length range: {min(sequence_lengths)}-{max(sequence_lengths)} bp")

### QC Metric 6: Sequence Duplication Levels

In [ ]:
# Calculate duplication levels
total_sequences = len(sequence_counts)
unique_sequences = sum(1 for count in sequence_counts.values() if count == 1)
duplication_rate = (1 - unique_sequences / total_sequences) * 100

# Group by duplication level
dup_levels = Counter()
for seq, count in sequence_counts.items():
    if count == 1:
        dup_levels['1'] += 1
    elif count == 2:
        dup_levels['2'] += 1
    elif count <= 4:
        dup_levels['3-4'] += 1
    elif count <= 9:
        dup_levels['5-9'] += 1
    elif count <= 49:
        dup_levels['10-49'] += 1
    else:
        dup_levels['50+'] += 1

# Status
if duplication_rate < 20:
    status = "PASS"
    color = "green"
elif duplication_rate < 50:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Plot
plt.figure(figsize=(12, 6))
levels = ['1', '2', '3-4', '5-9', '10-49', '50+']
counts = [dup_levels[l] for l in levels]
plt.bar(levels, counts, edgecolor='black', alpha=0.7)
plt.xlabel('Duplication Level')
plt.ylabel('Number of Sequences')
plt.title(f'Sequence Duplication Levels - {status}', color=color, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"Status: {status}")
print(f"Total sequences: {total_sequences:,}")
print(f"Unique sequences: {unique_sequences:,}")
print(f"Duplication rate: {duplication_rate:.1f}%")

### QC Metric 7: Overrepresented Sequences

In [ ]:
# Find most common sequences
most_common = sequence_counts.most_common(10)
threshold = read_count * 0.001  # 0.1% threshold

overrepresented = [(seq, count) for seq, count in most_common if count > threshold]

# Status
if len(overrepresented) == 0:
    status = "PASS"
    color = "green"
elif len(overrepresented) <= 3:
    status = "WARN"
    color = "orange"
else:
    status = "FAIL"
    color = "red"

# Display table
print(f"Status: {status}\n")
print(f"Overrepresented sequences (>{threshold:.0f} occurrences):")
print(f"{'Sequence':<50} {'Count':>10} {'Percentage':>12}")
print("-" * 75)

if overrepresented:
    for seq, count in overrepresented:
        pct = count / read_count * 100
        seq_display = seq[:47] + '...' if len(seq) > 50 else seq
        print(f"{seq_display:<50} {count:>10,} {pct:>11.2f}%")
else:
    print("No overrepresented sequences found.")

# Plot top 10
plt.figure(figsize=(12, 6))
seqs = [f"Seq {i+1}" for i in range(len(most_common))]
counts = [count for seq, count in most_common]
plt.bar(seqs, counts, edgecolor='black', alpha=0.7)
plt.axhline(y=threshold, color='red', linestyle='--', label=f'Threshold ({threshold:.0f})')
plt.xlabel('Sequence Rank')
plt.ylabel('Count')
plt.title(f'Top 10 Most Common Sequences - {status}', color=color, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### Quality Control Summary Report

**Understanding Quality Status:**

- **PASS** (Green): Meets quality thresholds - data is good to use
- **WARN** (Yellow): Slightly below optimal - review recommended but often acceptable
- **FAIL** (Red): Significant quality issues - data may need filtering or be unusable

**When to be concerned:**
- 1-2 WARN metrics: Normal for many datasets, review those specific metrics
- 3+ WARN metrics: Consider quality filtering before analysis
- Any FAIL metrics: Investigate the issue, may need different dataset or quality trimming

**Common WARN causes:**
- Per-base quality: Normal quality degradation at read ends
- GC content: Natural variation by organism
- Duplication: Expected in RNA-seq (highly expressed genes)

In [ ]:
# Generate summary report with actual status tracking
print("=" * 70)
print("QUALITY CONTROL SUMMARY REPORT")
print("=" * 70)
print(f"\nDataset: {accession}")
print(f"Total reads analyzed: {read_count:,}")
print(f"Average read length: {np.mean(sequence_lengths):.1f} bp")
print(f"Average quality score: {np.mean([np.mean(q) for q in quality_scores]):.1f}")
print(f"Average GC content: {np.mean(gc_contents):.1f}%")
print(f"\nQC Metrics Summary:")
print("-" * 70)
print(f"{'Metric':<45} {'Status':>10}")
print("-" * 70)

# Calculate statuses (these match the logic in each metric above)
median_quality = np.median([np.median(q) for q in quality_scores])
qc1_status = "PASS" if median_quality >= 28 else "WARN" if median_quality >= 20 else "FAIL"

peak_quality = np.percentile([np.mean(q) for q in quality_scores], 50)
qc2_status = "PASS" if peak_quality >= 30 else "WARN" if peak_quality >= 20 else "FAIL"

# Per-base content variation
positions = sorted(per_base_content.keys())
a_pct = [per_base_content[pos]['A'] / sum(per_base_content[pos].values()) * 100 for pos in positions]
max_variation = np.std(a_pct)
qc3_status = "PASS" if max_variation < 10 else "WARN" if max_variation < 20 else "FAIL"

std_gc = np.std(gc_contents)
qc4_status = "PASS" if std_gc < 10 else "WARN" if std_gc < 15 else "FAIL"

unique_lengths = len(Counter(sequence_lengths))
qc5_status = "PASS" if unique_lengths == 1 else "WARN" if unique_lengths <= 3 else "FAIL"

total_sequences = len(sequence_counts)
unique_sequences = sum(1 for count in sequence_counts.values() if count == 1)
duplication_rate = (1 - unique_sequences / total_sequences) * 100
qc6_status = "PASS" if duplication_rate < 20 else "WARN" if duplication_rate < 50 else "FAIL"

threshold = read_count * 0.001
overrepresented = [(seq, count) for seq, count in sequence_counts.most_common(10) if count > threshold]
qc7_status = "PASS" if len(overrepresented) == 0 else "WARN" if len(overrepresented) <= 3 else "FAIL"

# Print all statuses
metrics_status = [
    ("1. Per-base Quality Scores", qc1_status),
    ("2. Per-sequence Quality Distribution", qc2_status),
    ("3. Per-base Sequence Content", qc3_status),
    ("4. GC Content Distribution", qc4_status),
    ("5. Sequence Length Distribution", qc5_status),
    ("6. Sequence Duplication Levels", qc6_status),
    ("7. Overrepresented Sequences", qc7_status)
]

for metric, status in metrics_status:
    print(f"{metric:<45} {status:>10}")

# Overall assessment
pass_count = sum(1 for _, s in metrics_status if s == "PASS")
warn_count = sum(1 for _, s in metrics_status if s == "WARN")
fail_count = sum(1 for _, s in metrics_status if s == "FAIL")

print("\n" + "=" * 70)
print(f"Overall: {pass_count} PASS, {warn_count} WARN, {fail_count} FAIL")
print("=" * 70)
print("\nRecommendations:")
if fail_count > 0:
    print("  ⚠ FAIL metrics detected - review data quality before analysis")
if warn_count > 0:
    print("  ⚠ WARN metrics detected - consider quality filtering")
if pass_count == 7:
    print("  ✓ All metrics passed - data quality is excellent")
print("  • Review individual metrics above for detailed information")
print("  • Consider adapter trimming if overrepresented sequences detected")
print("  • Low quality bases may need trimming before downstream analysis")
print("=" * 70)

## Part 5: Cleanup Environment

To avoid incurring costs associated with the environment created in your account, delete the CloudFormation Stack which will remove all created resources. Execute the cells below to clean up local files and S3 buckets.

**Important**: If the S3 buckets created by CloudFormation are not empty, the Delete step for CloudFormation will fail.

### Delete Local Files

In [ ]:
# Clean up local files
import os
import shutil

print("Cleaning up local files...")

# Go back to parent directory
os.chdir('..')

# Remove sra_data directory
if os.path.exists('sra_data'):
    shutil.rmtree('sra_data')
    print("✓ Removed sra_data directory")
else:
    print("sra_data directory not found")

print("\nLocal cleanup complete!")

### Clean S3 Buckets

In [ ]:
# Comprehensive cleanup for versioned S3 buckets
def cleanup_versioned_bucket(bucket_name):
    """Delete all objects and versions from a versioned S3 bucket"""
    print(f"Starting cleanup of bucket: {bucket_name}")
    
    try:
        # First, list all current objects
        response = s3.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            print(f"Found {len(response['Contents'])} current objects")
            for obj in response['Contents']:
                print(f"  - {obj['Key']}")
        else:
            print("No current objects found")
        
        # List and delete all object versions
        paginator = s3.get_paginator('list_object_versions')
        pages = paginator.paginate(Bucket=bucket_name)
        
        delete_keys = []
        total_versions = 0
        total_markers = 0
        
        for page in pages:
            # Delete object versions
            if 'Versions' in page:
                for version in page['Versions']:
                    delete_keys.append({
                        'Key': version['Key'], 
                        'VersionId': version['VersionId']
                    })
                    total_versions += 1
            
            # Delete delete markers
            if 'DeleteMarkers' in page:
                for marker in page['DeleteMarkers']:
                    delete_keys.append({
                        'Key': marker['Key'], 
                        'VersionId': marker['VersionId']
                    })
                    total_markers += 1
        
        print(f"Found {total_versions} object versions and {total_markers} delete markers")
        
        if delete_keys:
            # Delete in batches of 1000 (AWS limit)
            deleted_count = 0
            for i in range(0, len(delete_keys), 1000):
                batch = delete_keys[i:i+1000]
                if batch:
                    response = s3.delete_objects(
                        Bucket=bucket_name,
                        Delete={'Objects': batch}
                    )
                    deleted_count += len(batch)
                    print(f"Deleted batch of {len(batch)} objects/versions")
            
            print(f"Successfully deleted {deleted_count} total objects/versions from {bucket_name}")
        else:
            print(f"No objects or versions to delete in {bucket_name}")
        
    except Exception as e:
        print(f"Error cleaning up {bucket_name}: {str(e)}")
        import traceback
        traceback.print_exc()

# Clean up both buckets
print("=" * 70)
print("S3 BUCKET CLEANUP")
print("=" * 70)
cleanup_versioned_bucket(dest_bucket)
print()

# Construct logging bucket name
account_id = dest_bucket.split('-')[0]
logging_bucket = f"{account_id}-opendata-sra-logs"
cleanup_versioned_bucket(logging_bucket)

print()
print("=" * 70)
print("CLEANUP COMPLETE")
print("=" * 70)
print("\nYou can now safely delete the CloudFormation stack.")
print("\nTo delete the stack, run:")
print("aws cloudformation delete-stack --stack-name opendata-sra-notebook-stack")